In [1]:
# wandb and pytorch manual 
# https://colab.research.google.com/drive/1XDtq-KT0GkX06a_g1MevuLFOMk4TxjKZ#scrollTo=bZpt5W2NNl6S

# for hyperparameter sweeping
# https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Organizing_Hyperparameter_Sweeps_in_PyTorch_with_W%26B.ipynb#scrollTo=W_YSZCkITmVJ


In [2]:
from __future__ import print_function
import os
import argparse
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# from torchvision import datasets, transforms

import logging
logging.propagate = False
logging.getLogger().setLevel(logging.ERROR)

import warnings
warnings.filterwarnings(action='ignore')

from datetime import datetime
import wandb

from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
from sklearn.metrics import f1_score, classification_report

In [3]:
!wandb login

wandb: Currently logged in as: beomgon-yu. Use `wandb login --relogin` to force relogin


## hyperparameter sweep using wandb

In [4]:
# use random search
sweep_config = {
    'method': 'random'
    }

# for bayesian search, this value should be included
metric = {
    'name': 'f1_score',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric    

parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        # 'values': ['adam']
        },
    # 'fc_layer_size': {
    #     'values': [128, 256, 512]
    #     },
    # 'dropout': {
    #       'values': [0.3, 0.4, 0.5]
    #     },
    }

parameters_dict.update({
    'epochs': {
        'value': 100}
    })

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'lr': {
        # a q_log_uniform_values distribution between 0 and 0.1
        'distribution': 'q_log_uniform_values',
        'q': 1e-5,
        'min': 1e-4,
        'max': 5e-2
      },
    'batch_size': {
        'values': [128,256,512,1024]
      },
    'svd_reduction': {
        'values': [True, False]
      },    
    # 'momentum': {
    #     # float between 0.1 and 0.9
    #     # with evenly-distributed logarithms 
    #     'distribution': 'uniform',
    #     'min': 0.1,
    #     'max': 0.9,
    #   },
    # 'weight_decay': {
    #     # float between 1e-3 and 1e-5
    #     # with evenly-distributed logarithms 
    #     'distribution': 'q_log_uniform_values',
    #     'q': 10,
    #     'min': 1e-5,
    #     'max': 1e-3,
    #   },    
    })

In [5]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'f1_score'},
 'parameters': {'batch_size': {'values': [128, 256, 512, 1024]},
                'epochs': {'value': 100},
                'lr': {'distribution': 'q_log_uniform_values',
                       'max': 0.05,
                       'min': 0.0001,
                       'q': 1e-05},
                'optimizer': {'values': ['adam', 'sgd']},
                'svd_reduction': {'values': [True, False]}}}


In [6]:
sweep_id = wandb.sweep(sweep_config, project="Credit Card Fraud Detection Sweep")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: nc9iukvl
Sweep URL: https://wandb.ai/beomgon-yu/Credit%20Card%20Fraud%20Detection%20Sweep/sweeps/nc9iukvl


In [7]:

# now = datetime.now().strftime('%Y%m%d_%H%M%S')
# wandb.init(project='Credit Card Fraud Detection',  name=now, mode='online')
# wandb.watch_called = False

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# config =wandb.config
# config.batch_size = 1024
# config.epochs = 100
# config.lr = 1e-2
# config.momentum = 0.5
# config.weight_decay = 1e-4
# config.device = device
# config.seed = 42
# config.log_interval = 10
# config.num_workers = 8
# config.adam = False


# set seed for reproduce

In [8]:
def seed_everything(seed) :
    random.seed(seed)
    os.environ['PYHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


## For SVD reduction

In [9]:
def reduced_mat(df) :
    u, s, v = np.linalg.svd(df, full_matrices=False)
    s[0] = 0
    rdf = pd.DataFrame( u @ np.diag(s) @ v)
    return rdf

# dataset

In [10]:
# Pandas is 18 times slower than Numpy (15.8ms vs 0.874 ms). Pandas is 20 times slower than Numpy
# therefore Use numpy for faster data loading

class CDataset(Dataset):
    def __init__(self, df, eval_mode=False):
        self.df = df
        self.eval_mode = eval_mode
        if self.eval_mode:
            self.labels = self.df['Class'].values
            self.df = self.df.drop(columns=['Class']).values
        else:
            self.df = self.df.values
        
    def __getitem__(self, index):
        if self.eval_mode:
            self.x = self.df[index]
            self.y = self.labels[index]
            return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x)
        
    def __len__(self):
        return len(self.df)

In [11]:
# train_dataset = CDataset(train_df)
# train_loader = DataLoader(train_dataset, batch_size = config.batch_size, shuffle=True, num_workers=config.num_workers)

# val_dataset = CDataset(val_df, eval_mode=True)
# val_loader = DataLoader(val_dataset, batch_size = config.batch_size, shuffle=False, num_workers=config.num_workers)

## model

In [12]:
class AutoEncoder(nn.Module) :
    def __init__(self) :
        super().__init__()
        # self.act = nn.ReLU()
        self.act = nn.GELU()
        # self.act = nn.LeakyReLU()
        
        self.dim = 30
        self.hidden1 = 64
        self.hidden2 = 128
        
        self.encoder = nn.Sequential(
            nn.BatchNorm1d(self.dim),
            nn.Linear(self.dim,self.hidden1),
            nn.BatchNorm1d(self.hidden1),
            self.act,
            nn.Linear(self.hidden1,self.hidden2),
            nn.BatchNorm1d(self.hidden2),
            self.act,
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(self.hidden2,self.hidden1),
            nn.BatchNorm1d(self.hidden1),
            self.act,
            nn.Linear(self.hidden1,self.dim),
        )
        
#         self.encoder2 = nn.Sequential(
#             nn.BatchNorm1d(self.dim),
#             nn.Linear(self.dim,self.hidden1),
#             nn.BatchNorm1d(self.hidden1),
#             self.act,
#             nn.Linear(self.hidden1,self.hidden2),
#             nn.BatchNorm1d(self.hidden2),
#             self.act,
#         )
        
#         self.decoder2 = nn.Sequential(
#             nn.Linear(self.hidden2,self.hidden1),
#             nn.BatchNorm1d(self.hidden1),
#             self.act,
#             nn.Linear(self.hidden1,self.dim),
#         )        
        
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)          
        
    def forward(self, x) :
        
        x = self.encoder(x)
        x = self.decoder(x)
        
#         x = self.encoder2(x1)
#         x = self.decoder2(x)
        
        return x
        

In [13]:
class Trainer() :
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, config) :
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = config.device
        self.epochs = config.epochs
        self.lr = config.lr
        
        self.criterion = nn.L1Loss().to(self.device)
        
    def fit(self,) :
        self.model.to(self.device)
        best_score = 0
        for epoch in range(self.epochs) :
            self.model.train()
            train_loss = []
            
            for x in iter(self.train_loader) :
                x = x.to(self.device)
                _x = self.model(x)
                
                loss = self.criterion(x, _x)
                # loss = self.criterion(x, _x) + 0.5*self.criterion(x, x1)
                
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
                train_loss.append(loss.item())
                wandb.log({'train loss' : loss.item()})
                
            score = self.validation(self.model, 0.95)
            wandb.log({'f1_score' : score})
            
            if self.scheduler is not None :
                self.scheduler.step(score)
            
            # print(f'epoch :[{epoch}] train loss [{np.mean(train_loss)}] val score [{score}]')
            
            # for param_group in self.optimizer.param_groups:
            #     print(param_group['lr'])      
            
            # print(f'epoch :[{epoch}] train loss [{np.mean(train_loss)}] val score [{score}] lr [{self.scheduler.get_lr()}]')

            if best_score < score :
                best_score = score
                
#             wandb.log({'best_score' : best_score})
# #                 torch.save(self.model.state_dict(), '../saved/best_model.pth')
            
    def validation(self, eval_model, threshold) :
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        eval_model.eval()
        pred_y = []
        true_y = []
        
        with torch.no_grad():
            for x, y in iter(self.val_loader) :
                x = x.to(self.device)
                y = y.to(self.device)
                
                _x = self.model(x)
                diff = cos(x, _x).cpu().tolist()
                batch_pred = np.where(np.array(diff) < threshold, 1, 0).tolist()
                pred_y += batch_pred
                true_y += y.tolist()

        return f1_score(true_y, pred_y, average='macro')

In [14]:

def main() :
    
    with wandb.init(config=None) :
        
        config = wandb.config
        config.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        config.num_workers = 8
        config.seed = 42
        config.momentum = 0.9
        config.weight_decay = 1e-4

        train_df = pd.read_csv('../dataset/train.csv')
        val_df = pd.read_csv('../dataset/val.csv')
        train_df = train_df.drop(columns=['ID'])
        val_df = val_df.drop(columns=['ID'])  
        
        if config.svd_reduction :
            train_df = reduced_mat(train_df)
            val = reduced_mat(val_df.drop(columns=['Class']))
            val['Class'] = val_df['Class']
            val_df = val

        train_dataset = CDataset(train_df)
        train_loader = DataLoader(train_dataset, batch_size = config.batch_size, shuffle=True, num_workers=config.num_workers)

        val_dataset = CDataset(val_df, eval_mode=True)
        val_loader = DataLoader(val_dataset, batch_size = config.batch_size, shuffle=False, num_workers=config.num_workers)    

        seed_everything(config.seed)    

        model = AutoEncoder()
        model.eval()

        # if config.adam :
        if config.optimizer == 'adam' :
            optimizer = torch.optim.Adam(params=model.parameters(), lr = config.lr)
        else :
            optimizer = torch.optim.SGD(model.parameters(), config.lr,
                                        momentum=config.momentum,
                                        weight_decay=config.weight_decay)

        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs', min_lr=1e-8, verbose=True)
        # scheduler = StepLR(optimizer, step_size=50, gamma=0.2)

        wandb.watch(model, log='all')

        trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, config)

        wandb.save('model.h5')

        trainer.fit()

In [15]:
# if __name__ == '__main__':
#     main(config)

In [16]:
wandb.agent(sweep_id, main, count=20)

wandb: Agent Starting Run: pj39658o with config:
wandb: 	batch_size: 1024
wandb: 	epochs: 100
wandb: 	lr: 0.026310000000000004
wandb: 	optimizer: adam
wandb: 	svd_reduction: True
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: beomgon-yu. Use `wandb login --relogin` to force relogin


Epoch    24: reducing learning rate of group 0 to 1.3155e-02.
Epoch    35: reducing learning rate of group 0 to 6.5775e-03.
Epoch    46: reducing learning rate of group 0 to 3.2888e-03.
Epoch    57: reducing learning rate of group 0 to 1.6444e-03.
Epoch    68: reducing learning rate of group 0 to 8.2219e-04.
Epoch    79: reducing learning rate of group 0 to 4.1109e-04.
Epoch    90: reducing learning rate of group 0 to 2.0555e-04.


f1_score,▁▄▅▆███▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train loss,▆▄▄█▄▃▆▃▃▄▃▃▃▃▂▂▂▃▂▄▂▃▄▁▂▄▁▂▂▁▁▁▃▁▁▁▁▁▄▁
f1_score,0.49973
train loss,0.08523


wandb: Agent Starting Run: 4zzm2za1 with config:
wandb: 	batch_size: 512
wandb: 	epochs: 100
wandb: 	lr: 0.004580000000000001
wandb: 	optimizer: adam
wandb: 	svd_reduction: False
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch    44: reducing learning rate of group 0 to 2.2900e-03.
Epoch    55: reducing learning rate of group 0 to 1.1450e-03.
Epoch    66: reducing learning rate of group 0 to 5.7250e-04.
Epoch    77: reducing learning rate of group 0 to 2.8625e-04.
Epoch    88: reducing learning rate of group 0 to 1.4313e-04.
Epoch    99: reducing learning rate of group 0 to 7.1563e-05.


...

Epoch    27: reducing learning rate of group 0 to 1.1000e-03.
Epoch    38: reducing learning rate of group 0 to 5.5000e-04.
Epoch    49: reducing learning rate of group 0 to 2.7500e-04.
Epoch    60: reducing learning rate of group 0 to 1.3750e-04.
Epoch    71: reducing learning rate of group 0 to 6.8750e-05.
Epoch    82: reducing learning rate of group 0 to 3.4375e-05.
Epoch    93: reducing learning rate of group 0 to 1.7188e-05.


f1_score,▁▅▄▇▅▆█▅▇█████▇▇▅▇▇▇▇▇▇▇▇▆▇▇▇▇▃▇▇▇▇▆▇▇▇▇
train loss,█▄▄█▄▄▃▃▄▃▇▃▂▂▂▂▄▂▂▂▂▂▂▂▃▂▂▃▃▂▁▁▄▂▁▆▂▂▃▁
f1_score,0.79207
train loss,0.08613


wandb: Agent Starting Run: iman4m3h with config:
wandb: 	batch_size: 1024
wandb: 	epochs: 100
wandb: 	lr: 0.00054
wandb: 	optimizer: sgd
wandb: 	svd_reduction: False
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch    12: reducing learning rate of group 0 to 2.7000e-04.
Epoch    23: reducing learning rate of group 0 to 1.3500e-04.
Epoch    34: reducing learning rate of group 0 to 6.7500e-05.
Epoch    45: reducing learning rate of group 0 to 3.3750e-05.
Epoch    56: reducing learning rate of group 0 to 1.6875e-05.
Epoch    67: reducing learning rate of group 0 to 8.4375e-06.
Epoch    78: reducing learning rate of group 0 to 4.2188e-06.
Epoch    89: reducing learning rate of group 0 to 2.1094e-06.
Epoch   100: reducing learning rate of group 0 to 1.0547e-06.


f1_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train loss,█▇▆▄▄▃▃▂▂▂▂▂▁▂▂▂▁▁▁▂▁▂▂▁▂▁▁▁▂▁▁▁▂▁▁▁▁▁▂▁
f1_score,0.00105
train loss,0.61812


In [20]:
a = torch.tensor(1).cuda()
b = torch.tensor(0)

In [18]:
a

tensor(1, device='cuda:0')

In [21]:
print(a.device)
print(b.device)


cuda:0
cpu
